In [1]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_dir = './FMCG/train/images'
csv_data = pd.read_csv('./FMCG/train/_classes.csv')
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [3]:
csv_data.columns

Index(['filename', ' AfterShave', ' FabricBrightener', ' FaceMask',
       ' Handwash', ' Lotion', ' Napkin', ' Oil', ' PetroliumJelly', ' Powder',
       ' Soap', ' Sweetner', ' Toothpaste'],
      dtype='object')

In [4]:
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(
    dataframe=csv_data,
    directory=train_dir,
    x_col='filename',
    y_col=[' AfterShave', ' FabricBrightener', ' FaceMask', ' Handwash', ' Lotion', ' Napkin', ' Oil', ' PetroliumJelly', ' Powder', ' Soap', ' Sweetner', ' Toothpaste'],  # Keep the spaces
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',
    shuffle=True
)

Found 202 validated image filenames.


In [5]:
from tensorflow.keras import regularizers

# Define a Sequential model
model = tf.keras.models.Sequential([
    # Flatten the input to feed it into the dense layers
    tf.keras.layers.Flatten(input_shape=(128, 128, 3)),  # Input shape for dense network

    # First dense layer
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.5),  # Dropout layer to reduce overfitting

    # Second dense layer
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.5),

    # Third dense layer
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.5),

    # Output layer for 12 classes
    tf.keras.layers.Dense(12, activation='sigmoid')
])


C:\Users\MANN\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
test_dir = './FMCG/test/images'  # Path to test images
test_csv = './FMCG/test/_classes.csv'  # CSV containing test filenames and labels

test_data = pd.read_csv(test_csv)

In [7]:
test_generator = datagen.flow_from_dataframe(
    dataframe=test_data,
    directory=test_dir,
    y_col=[' AfterShave', ' FabricBrightener', ' FaceMask', ' Handwash', ' Lotion', ' Napkin', ' Oil', ' PetroliumJelly', ' Powder', ' Soap', ' Sweetner', ' Toothpaste'],  # Keep the spaces
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',  
    shuffle=False 
)

Found 24 validated image filenames.


In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=25
)

Epoch 1/25


C:\Users\MANN\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 146ms/step - accuracy: 0.0654 - loss: 21.1737
Epoch 2/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 144ms/step - accuracy: 0.1132 - loss: 18.4327
Epoch 3/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 145ms/step - accuracy: 0.0841 - loss: 14.2136
Epoch 4/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 146ms/step - accuracy: 0.0911 - loss: 10.6178
Epoch 5/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 145ms/step - accuracy: 0.0857 - loss: 8.7316
Epoch 6/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 145ms/step - accuracy: 0.0678 - loss: 7.3025
Epoch 7/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 145ms/step - accuracy: 0.0811 - loss: 6.3827
Epoch 8/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 146ms/step - accuracy: 0.0924 - loss: 5.6654
Epoch 9/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 145ms/step - accuracy: 0.1090 - loss: 5.0172
Epoch 10/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 145ms/step - accuracy: 0.0940 - loss: 4.5198
Epoch 11/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 146ms/step - accuracy: 0.1034 - loss: 4.1862
Epoch 12/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 145ms/step - accuracy: 0.1231 - loss: 3.8

In [9]:
loss, accuracy = model.evaluate(test_generator)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step - accuracy: 0.0833 - loss: 2.5080
Test Loss: 2.507962226867676, Test Accuracy: 0.0833333358168602


C:\Users\MANN\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


In [10]:
predictions = model.predict(test_generator)
predicted_classes = predictions.argmax(axis=-1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step


In [11]:
class_labels = [' AfterShave', ' FabricBrightener', ' FaceMask', ' Handwash', ' Lotion', ' Napkin', ' Oil', ' PetroliumJelly', ' Powder', ' Soap', ' Sweetner', ' Toothpaste']  # Keep the spaces
    

predicted_class_names = [class_labels[predicted_index] for predicted_index in predicted_classes]

for filename, predicted_class in zip(test_data['filename'], predicted_class_names):
    print(f'Filename: {filename}, Predicted Class: {predicted_class}')

Filename: images-1-_jpeg.rf.27fd9e5aa4f310481f20e23895d9d15c.jpg, Predicted Class:  PetroliumJelly
Filename: download-5-_jpeg.rf.9b69ce4dbbb2058f38d42ec41caeb398.jpg, Predicted Class:  PetroliumJelly
Filename: images_jpeg.rf.6a81f460b00de4513ede32c01948f2fb.jpg, Predicted Class:  PetroliumJelly
Filename: hq720_jpg.rf.452a8276c4b12dd07d2c472c2b332ddd.jpg, Predicted Class:  PetroliumJelly
Filename: download_jpeg.rf.b3edaf1cfb9702849cf424e9b5b3ee44.jpg, Predicted Class:  PetroliumJelly
Filename: download-9-_jpeg.rf.9de97aa023aa35128ba21b33160d83ae.jpg, Predicted Class:  PetroliumJelly
Filename: 8941100501214_jpg.rf.9e879b8637215a9e7b9a8fcd4ba4a70a.jpg, Predicted Class:  PetroliumJelly
Filename: chamak-top-img-new-update-2023_png.rf.3519ad81a56c7b9c87895fd3bd19c9f6.jpg, Predicted Class:  PetroliumJelly
Filename: OOJ44hHIeX6fowKcTrMoQj56fXi7xkZiRl7FvqkA_jpg.rf.c7ad5a7ce486556eefd9563745559c69.jpg, Predicted Class:  PetroliumJelly
Filename: 8def6e4622fca704e114c6785f3013dd_jpg.rf.1de911766e4

In [29]:
model.save('FMCG.h5')